Calculate heritability for each metabolite as in [Breunig JS, Hackett SR, Rabinowitz JD, Kruglyak L (2014) Genetic Basis of Metabolome Variation in Yeast. PLoS Genet 10(3): e1004142](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1004142)

"For each metabolite, segregants with two quantifiable biological replicates were isolated and the variance within replicates was compared to the total across all samples. This is effectively subtracting the environmental variance from the total phenotypic variance to yield the genetic variance. The ratio of genetic variance to phenotypic variance is the broad sense heritability (equation 1)"

$$\hat\sigma^2_s=\frac{\sum_{r=1}^2 (X_{sr}-\bar X_s)^2}{2} . \frac{2}{2-1} $$

$$H^2=1-\frac{\sum_s^S 2 \hat\sigma_s^2}{\sum_s^S \sum_{r=1}^2 (X_{sr}-\bar X)^2} $$

In [5]:
# config opts and libraries
options(repr.plot.width = 6)
options(repr.plot.height = 5)
library(ggplot2);
library(plyr);
library(dplyr);
library(reshape2);
library(LSD);
library(qtl);
library(pheatmap);
library(funqtl);
library(parallel);
options(mc.cores = 24);
library(snow);
library(igraph);
library(stringr);
library(RColorBrewer);

In [18]:
# load data
endo_f = "/g/steinmetz/project/GenPhen/data/endometabolome/data/endometabolite_full_12102015.rda"
load(endo_f)
#head(endometabolite)

In [17]:
tmp = endometabolite %>% filter(time_format == "relative", metabolite == "AKG")
length(unique(endometabolite$strain))

[1] 104

In [31]:
all_var = endometabolite %>% filter(time_format == "relative", time == 1) %>% 
    group_by(metabolite, time) %>% do({
        metabolite = .$metabolite[1]
        time = .$time[1]
        big_mean = mean(.$value, na.rm = T)
        df = data.frame(metabolite = metabolite, time = time, mean = big_mean)
        return(df)
})
all_var

,metabolite,time,mean
1,AKG,1,28.12393
2,CAN,1,12.32951
3,CIT,1,2238.648
4,GLUC,1,265.5263
5,FUM,1,68.27573
6,MAL,1,364.2321
7,PEP,1,18.05307
8,PYR,1,716.4063
9,SUC,1,144.9005
10,ALA,1,1817.228


In [7]:
# use relative time_format
strain_var = endometabolite %>% filter(time_format == "relative") %>% 
    group_by(metabolite, time, strain) %>% do({
        metabolite = .$metabolite[1]
        strain = .$strain[1]
        time = .$time[1]
        small_mean = mean(.$value)
        var = (sum((.$value - small_mean)^2)/2) * 2
        #var_big = sum((.$value - filter(all_var, metabolite == metabolite, time == time)$mean)^2)
        df = data.frame(metabolite = metabolite, strain = strain, time = time, var = var)
        return(df)
})

Warning message:
In rbind_all(out[[1]]): Unequal factor levels: coercing to character

In [ ]:
H2 = strain_var %>% group_by(., metabolite, time) %>% do({
    sum(.$var,na.rm=T)
})

In [30]:
head(strain_var)

,metabolite,strain,time,var
1,AKG,01B,1,0.1076056
2,AKG,01C,1,0.3629404
3,AKG,01D,1,17.87012
4,AKG,02C,1,77.27674
5,AKG,02D,1,3.86483
6,AKG,03A,1,5.022146
